In [1]:
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import optimizers
import re
import nltk
import snowballstemmer
stemmer = snowballstemmer.stemmer('tamil')
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy

In [2]:
from datasets import load_dataset,list_datasets
a=load_dataset("ai4bharat/naamapadam","ta")

Found cached dataset naamapadam (C:/Users/dhanu/.cache/huggingface/datasets/ai4bharat___naamapadam/ta/1.0.0/c1b045180d60b208d2468bdad897d04461f08c7137c04a85220697b1bef7df9a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
print(a)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 497882
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 758
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2795
    })
})


In [4]:
idx=1
rec=a['train'][5]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))

நல்கொண்டா	5
பகுதியிலேயே	0
இந்த	0
சம்பவம்	0
பதிவாகியுள்ளது	0
.	0


In [5]:
column_names = a["train"].column_names
print(column_names)

features = a["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [6]:
iob_tag={0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}

In [7]:
a["train"]

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 497882
})

In [8]:
def data_list(data):
    x,y=[],[]
    for i in data:
        x.append(i['tokens'])
        t1=[]
        for j in i["ner_tags"]:
            t1.append(iob_tag[j])
        y.append(t1)
    return x,y

x_train,y_train=data_list(a["train"])
x_test,y_test=data_list(a["test"])
x_val,y_val=data_list(a["validation"])

In [9]:
def pre_process(data,y):
    l=[]
    l1=[]
    for i,j in zip(data,y):
        t=[]
        t1=[]
        for k,m in zip(i,j):
            x=re.sub("\W","",k)
            if x=="":
                continue
            t.append(x)
            t1.append(m)
        l.append(t)
        l1.append(t1)
    return l,l1
x_train,y_train=pre_process(x_train,y_train)
x_test,y_test=pre_process(x_test,y_test)
x_val,y_val=pre_process(x_val,y_val)

In [10]:
# prepare english tokenizer
def tokenize(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    size_vocab= len(tokenizer.word_index) + 1
    return tokenizer,size_vocab

x_train_token,x_train_size=tokenize(x_train)
y_train_token,y_train_size=tokenize(y_train)
x_test_token,x_test_size=tokenize(x_test)
y_test_token,y_test_size=tokenize(y_test)
x_val_token,x_val_size=tokenize(x_val)
y_val_token,y_val_size=tokenize(y_val)

In [11]:
def encode(tokenizer, m, data):
    seq = tokenizer.texts_to_sequences(data)
    seq = pad_sequences(seq, maxlen=m, padding='post')
    return seq

m=0
data=x_train+x_val+x_test
for i in data:
    if m<len(i):
        m=len(i)
print("max length:",m)

X_train = encode(x_train_token, m, x_train)
Y_train= encode(y_train_token, m, y_train)
X_test = encode(x_test_token, m, x_test)
Y_test= encode(y_test_token, m, y_test)
X_val = encode(x_val_token, m, x_val)
Y_val= encode(y_val_token, m, y_val)

max length: 91


In [20]:
    #encoder
    model = Sequential()
    model.add(Embedding(x_train_size, units, input_length=in_timesteps, mask_zero=True))
    model.add(LSTM(units))
    #decoder
    model.add(RepeatVector(out_timesteps))
    model.add(LSTM(units, return_sequences=True))
    model.add(Dense(y_train_size, activation='softmax'))
    model.summary()
    rms = optimizers.RMSprop(lr=0.001)
    model.compile(optimizer=rms, loss='sparse_categorical_crossentropy',metrics=["accuracy"])
    history = model.fit(X_train, Y_train, epochs=2, validation_data = (X_val,Y_val), verbose=1)
    return model,history

In [21]:
model,history = model(x_train_size,y_train_size , m, m, 64)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 91, 64)            22871552  
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 repeat_vector_2 (RepeatVect  (None, 91, 64)           0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 91, 64)            33024     
                                                                 
 dense_2 (Dense)             (None, 91, 8)             520       
                                                                 
Total params: 22,938,120
Trainable params: 22,938,120
Non-trainable params: 0
__________________________________________

In [22]:
model.save("nlp.h5")
densenet=load_model("nlp.h5")
p=densenet.predict(X_test)

24/24 [==============================] - 2s 23ms/step


In [23]:
model.evaluate(X_test,Y_test)

24/24 [==============================] - 1s 22ms/step - loss: 0.1862 - accuracy: 0.9609


[0.18620574474334717, 0.9608715772628784]